In [ ]:
import os
import sys
import pandas as pd
import ipywidgets as widgets
from IPython.core.display import display, HTML
from IPython.display import display, IFrame, Image

display(HTML('<link rel="stylesheet" href="style_browser.css">'))

from ucsfneuroviz.importer import import_dataframe
from ucsfneuroviz.interactive_behav_plots import interactive_individual_line_plot
from ucsfneuroviz.interactive_struct_plots import interactive_brain_zscore_plot, validate_id_number
from ucsfneuroviz.interactive_diffusion_plots import interactive_dti_metrics

from ucsfneuroviz.fc_vars import FC_vars, FC_vars_select

# path='/Volumes/language/language/mkersey/qsiprep_related/testing_recon_options/mrtrix_pyafq_tractometry_reco/sub-34429/ses-20230317/dwi/sub-34429_ses-20230317_space-T1w_desc-preproc/sub-34429_ses-20230317_space-T1w_desc-preproc_dwi_space-RASMM_model-probCSD_algo-AFQ_desc-viz_dwi.html'
# # add path to working directory
# sys.path.append(path)

# iframe = IFrame(src=path, width="100%", height="600")
# display(iframe)

In [ ]:
# Insert photo from ./data/ folder
Image(filename='./data/UCSF_sublogo_DyslexiaCenter_navy_RGB.png', width=250)

In [ ]:
# Import Data
brain_df = pd.read_csv('/Volumes/language/language/math_cognition/ucsfneuroviz/structural/all_structural_data.csv', index_col=0)
behavior_df = import_dataframe('/Volumes/language/language/math_cognition/ucsfneuroviz/behavior/redcap_labels_with_diagnoses.csv')
profiles_df = pd.read_csv('/Volumes/language/language/math_cognition/ucsfneuroviz/diffusion/all_profiles_data.csv', index_col=0)
slcount_df = pd.read_csv('/Volumes/language/language/math_cognition/ucsfneuroviz/diffusion/all_slcount_data.csv', index_col=0)

# Convert 'Rey-O Delay Percentile' "<1" to 0.5 and Force to numeric
behavior_df['Rey-O Delay Percentile:'] = behavior_df['Rey-O Delay Percentile:'].apply(lambda x: 0.5 if x =="<1" else x)
behavior_df['Rey-O Delay Percentile:'] = pd.to_numeric(behavior_df['Rey-O Delay Percentile:'])
# Replace all instances of "Unchecked" with 0 and "Checked" with 1
# behavior_df = behavior_df.replace(to_replace="Unchecked", value=0)
# behavior_df = behavior_df.replace(to_replace="Checked", value=1)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Initialize text input widgets for subject ID and date
subject_id_input = widgets.Text(
    value='',
    placeholder='Enter Subject ID',
    description='Subject ID:',
    disabled=False
)

date_input = widgets.Text(
    value='',
    placeholder='Enter Date',
    description='Date:',
    disabled=False
)

submit_button = widgets.Button(
    description='Submit',
    disabled=False,
    button_style='',
    tooltip='Click to generate plots',
    icon='check'
)

# Function to generate all outputs
def generate_all_outputs(button):
    # Clear error output
    out_error.clear_output(wait=True)
    for i in range(len(tab.children)):
        generate_outputs(i)

# Attach click event to the submit button
submit_button.on_click(generate_all_outputs)

# Display the text input widgets and the submit button
input_widgets = widgets.HBox([subject_id_input, date_input, submit_button])
display(input_widgets)

# Initialize output widgets for each tab
structural_output = widgets.Output()
diffusion_output = widgets.Output()
functional_output = widgets.Output()
behavior_output = widgets.Output()
out_error = widgets.Output()

display(out_error)

# Initialize the tab widget and set the titles
tab = widgets.Tab()
tab.children = [structural_output, diffusion_output, functional_output, behavior_output]
tab.set_title(0, 'Structural MRI')
tab.set_title(1, 'Diffusion MRI')
tab.set_title(2, 'Functional MRI')
tab.set_title(3, 'Behavioral Scores')

diagnosis_columns = ['Diagnosis Overall', 'Dyslexia and ADHD', 'Dyslexia Phenotype', 'Dyslexia Phenotype Short', 'Dyslexia and Dyscalculia', 'Dyslexia and MLD', 'MLD Subtype Primary', 'MLD Subtype Secondary']

# Define your function to generate outputs based on the selected tab
def generate_outputs(tab_index):
    # Fetch subject ID and date from the global text input widgets
    # subject_id = subject_id_input.value.strip()
    subject_id = validate_id_number(subject_id_input.value.strip(), behavior_df, out_error)
    date = date_input.value.strip()

    if not subject_id or not date:
        print("Please enter both Subject ID and Date.")
        return

    if tab_index == 0:  # Structural MRI
        with structural_output:
            structural_output.clear_output(wait=True)
            interactive_brain_zscore_plot(brain_df, behavior_df, diagnosis_columns, subject_id, date)

    elif tab_index == 1:  # Diffusion MRI
        with diffusion_output:
            diffusion_output.clear_output(wait=True)

            local_path = "./data/" # local folder to copy file to
            ldrive_path = f"/Volumes/language/language/Dyslexia_project/BIDS/derivatives/QSIPrep/qsirecon/html/"
            file_name = f"sub-{subject_id}_ses-{date}_space-T1w_desc-preproc_dwi_space-RASMM_model-probCSD_algo-AFQ_desc-viz_dwi.html"
            full_path_to_file = os.path.join(ldrive_path, file_name)

            # if file is not already in local folder, copy it from ldrive
            if not os.path.isfile(os.path.join(local_path, file_name)):
                os.system(f"cp {full_path_to_file} {local_path}")

            full_path_local = os.path.join(local_path, file_name)

            iframe = IFrame(src=full_path_local, width="100%", height="600")
            display(iframe)

            interactive_dti_metrics(slcount_df, profiles_df, behavior_df, diagnosis_columns, sub_id=subject_id, ses_id=date)
    
    elif tab_index == 2:  # Functional MRI
        with functional_output:
            functional_output.clear_output(wait=True)
            local_path = "./data/" # local folder to copy file to
            ldrive_path_func = f"/Volumes/language/language/Dyslexia_project/BIDS/derivatives/fMRIPrep/html/faces-oth/"
            file_name_func = f"sub-{subject_id}_ses-{date}_alpha0.05_interactive_floc_contrast_faces-oth.html"
            full_path_to_file_func = os.path.join(ldrive_path_func, file_name_func)

            # if file is not already in local folder, copy it from ldrive
            if not os.path.isfile(os.path.join(local_path, file_name_func)):
                os.system(f"cp {full_path_to_file_func} {local_path}")

            full_path_local_func = os.path.join(local_path, file_name_func)

            iframe = IFrame(src=full_path_local_func, width="100%", height="600")
            display(iframe)

    elif tab_index == 3:  # Behavioral Scores
        with behavior_output:
            behavior_output.clear_output(wait=True)
            interactive_individual_line_plot(behavior_df, 'ID Number', diagnosis_columns, FC_vars_select, subject_id, date)

# Function to run when the selected tab changes
def on_tab_value_change(change):
    generate_outputs(change['new'])  # Manually trigger the function

# Attach the change event to the tab
tab.observe(on_tab_value_change, names='selected_index')

# Generate default outputs (for the first tab)
# generate_outputs(0)

# Display the tab
display(tab)



In [ ]:
# diagnosis_columns = ['Diagnosis Overall', 'Dyslexia and ADHD', 'Dyslexia Phenotype', 'Dyslexia Phenotype Short', 'Dyslexia and Dyscalculia', 'Dyslexia and MLD', 'MLD Subtype Primary', 'MLD Subtype Secondary']
# interactive_brain_zscore_plot(brain_df, behavior_df, diagnosis_columns)

In [ ]:
# interactive_individual_line_plot(behavior_df, 'ID Number', diagnosis_columns, FC_vars_select)

/var/folders/9r/l5byf96j5qv7bx1hlzlkwz7h0000gp/T/ipykernel_42581/3730426192.py:9: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  scores = subject_row[FC_vars_select].values[0]


KeyError: "None of [Index(['WASI Matrix Reasoning', 'WISC-IV Digit Span Forward',\n       'WISC-IV Digit Span Backward', 'NEPSY-II Naming',\n       'Children's Colored Trails A', 'Rey-O Delay',\n       'Judgment of Line Orientation', 'NEPSY-II Semantic Fluency',\n       'NEPSY-II Phonemic Fluency', 'NEPSY-II Inhibition',\n       'NEPSY-II Switching', 'Children's Colored Trails B', 'WCJ Segmentation',\n       'WCJ Sound Blending', 'WCJ Sound Awareness', 'WCJ Nonword Repetition',\n       'WCJ Memory for Words', 'WCJ Sentence Repetition',\n       'WCJ Rapid Picture Naming', 'WCJ Letter-Word Identification',\n       'WCJ Word Attack', 'TOWRE-2 Sight Word Efficiency',\n       'TOWRE-2 Phonemic Decoding Efficiency', 'GORT-5 Rate',\n       'GORT-5 Accuracy', 'GORT-5 Comprehension'],\n      dtype='object')] are in the [columns]"